In [0]:
import torch
from torch import nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import pickle

from pathlib import Path

import matplotlib.pyplot as plt

import pandas as pd

from fastai.basic_train import Learner, Callback
from fastai.basic_data import DataBunch
from fastai.vision.data import ImageList
from fastai.vision.models import resnet50, resnet18
from fastai.metrics import accuracy

import torchvision.transforms as transforms

import os
import glob
import shutil
import random

In [0]:
# Courtesy of https://forums.fast.ai/t/how-does-one-download-imagenet/40660/9
!wget http://files.fast.ai/data/imagenet-sample-train.tar.gz

--2020-04-06 23:28:07--  http://files.fast.ai/data/imagenet-sample-train.tar.gz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2232645974 (2.1G) [application/gzip]
Saving to: ‘imagenet-sample-train.tar.gz’

imagenet-sample-tra 100%[===================>]   2.08G  41.2MB/s    in 23s     

2020-04-06 23:28:30 (93.5 MB/s) - ‘imagenet-sample-train.tar.gz’ saved [2232645974/2232645974]



In [0]:
!wget http://files.fast.ai/data/imagenet-csv.tgz

--2020-04-06 23:28:35--  http://files.fast.ai/data/imagenet-csv.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4549635 (4.3M) [application/x-gtar-compressed]
Saving to: ‘imagenet-csv.tgz’

imagenet-csv.tgz    100%[===================>]   4.34M  --.-KB/s    in 0.09s   

2020-04-06 23:28:35 (46.3 MB/s) - ‘imagenet-csv.tgz’ saved [4549635/4549635]



In [0]:
!tar -xvf imagenet-sample-train.tar.gz

Streaming output truncated to the last 5000 lines.
train/n04259630/n04259630_11233.JPEG
train/n02113186/
train/n02113186/n02113186_4618.JPEG
train/n02113186/n02113186_2010.JPEG
train/n02113186/n02113186_12627.JPEG
train/n02113186/n02113186_8336.JPEG
train/n02113186/n02113186_10431.JPEG
train/n02113186/n02113186_2861.JPEG
train/n02113186/n02113186_961.JPEG
train/n02113186/n02113186_7075.JPEG
train/n02113186/n02113186_11222.JPEG
train/n02113186/n02113186_13841.JPEG
train/n02113186/n02113186_4198.JPEG
train/n02113186/n02113186_704.JPEG
train/n02113186/n02113186_7634.JPEG
train/n02113186/n02113186_1969.JPEG
train/n02113186/n02113186_19460.JPEG
train/n02113186/n02113186_12553.JPEG
train/n02113186/n02113186_9934.JPEG
train/n02113186/n02113186_10447.JPEG
train/n02113186/n02113186_5248.JPEG
train/n02113186/n02113186_632.JPEG
train/n02113186/n02113186_3456.JPEG
train/n02113186/n02113186_4784.JPEG
train/n02113186/n02113186_14760.JPEG
train/n02113186/n02113186_5457.JPEG
train/n02113186/n02113186_

In [0]:
reference_fraction = 0.1
all_files = glob.glob("train/*/*")
random.seed(420)
random.shuffle(all_files)
reference = all_files[:int(reference_fraction * len(all_files))]
reference_dir_name = "reference"
os.makedirs(reference_dir_name, exist_ok=True)
for path in reference:
  new_path = path.split("/")
  new_path[0] = reference_dir_name
  new_path = "/".join(new_path)
  os.makedirs(os.path.dirname(new_path), exist_ok=True)
  shutil.move(path, new_path)

In [0]:
!tar -xvf imagenet-csv.tgz

csv/
csv/valid.csv
csv/train.csv


In [0]:
labels = pd.read_csv("csv/train.csv")

In [0]:
labels.head()

,x,y
0,train/n07754684/n07754684_1900.JPEG,n07754684
1,train/n07754684/n07754684_27421.JPEG,n07754684
2,train/n07754684/n07754684_20089.JPEG,n07754684
3,train/n07754684/n07754684_34374.JPEG,n07754684
4,train/n07754684/n07754684_19961.JPEG,n07754684


In [0]:
image_size = 128

In [0]:
data = (ImageList.from_folder("train").
        split_by_rand_pct(0.2, seed = 567).
        label_from_folder(). #lambda x: os.path.basename(os.path.dirname(x))
        transform(size=image_size).
        databunch())